In [1]:
seed_list = [99843, 92935, 91761, 91361, 89580, 88921, 86725, 81843, 78995, 71347, 7023, 68128, 66694, 64348, 61880, 56857, 5403, 44830, 38229, 36240, 35205, 3345348918, 20168, 12919, 10290]
covmat_scaling_factor = 1.0

In [2]:
import numpy as np
import pathlib
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from validphys.api import API
from validphys.loader import FallbackLoader

%matplotlib inline

l = FallbackLoader()

In [3]:
#fitnames = [f"240715-ab-multict-fseed{seed}-tcm-nlo" for seed in seed_list]
fitnames = [f"240715-ab-multict-fseed{seed}-tcm-nlo" for seed in [10290, 3345348918]]
fits = [API.fit(fit=fitname) for fitname in fitnames]

# always the same so just take the first
prior_pdf = fits[0].as_input()["theorycovmatconfig"]["pdf"]

common_dicts = [dict(
    dataset_inputs={"from_": "fit"},
    fit=fit.name,
    fits=[fit.name],
    use_cuts="fromfit",
    metadata_group="nnpdf31_process",
) for fit in fits]

theoryids_dicts = [({
        "point_prescription": {"from_": "theorycovmatconfig"},
        "theoryids":{ "from_": "scale_variation_theories"},
        "t0theoryid": {"from_": "theory"},
        "theoryid": {"from_": "theory"},
        "theory": {"from_": "fit"},
        "theorycovmatconfig": {"from_": "fit"},
    } | common_dict) for common_dict in common_dicts]

# they are all the same so just take te first 
theoryids = API.theoryids(**theoryids_dicts[0])
theory_plus = theoryids[1].id
theory_mid = theoryids[0].id
theory_min = theoryids[2].id

# Inputs for central theory (used to construct the alphas covmat)
inps_centrals = [dict(theoryid=theory_mid, pdf=prior_pdf, **common_dict) for common_dict in common_dicts]

# Inputs for plus theory (used to construct the alphas covmat)
inps_pluss = [dict(theoryid=theory_plus, pdf=prior_pdf, **common_dict) for common_dict in common_dicts]

# Inputs for minus theory prediction (used to construct the alphas covmat)
inps_minuss = [dict(theoryid=theory_min, pdf=prior_pdf, **common_dict) for common_dict in common_dicts]

# inputs for the computation of the prediction of the fit with cov=C+S, where S is computed using the
# inps_central, inps_plus, and inps_minus dictionaries
inps_central_fits = [dict(theoryid=theory_mid, pdf={"from_": "fit"}, **common_dict) for common_dict in common_dicts]

prior_theorypreds_centrals = [API.group_result_table_no_table(**inps_central).iloc[:, 2:].mean(axis=1) for inps_central in inps_centrals]

prior_theorypreds_pluss = [API.group_result_table_no_table(**inps_plus).iloc[:, 2:].mean(axis=1) for inps_plus in inps_pluss]

prior_theorypreds_minuss = [API.group_result_table_no_table(**inps_minus).iloc[:, 2:].mean(axis=1) for inps_minus in inps_minuss]

gammas = [prior_theorypreds_plus + prior_theorypreds_minus - 2 * prior_theorypreds_central for prior_theorypreds_plus,prior_theorypreds_minus,prior_theorypreds_central in zip(prior_theorypreds_pluss,prior_theorypreds_minuss,prior_theorypreds_centrals) ]

LHAPDF 6.4.0 loading all 101 PDFs in set 240411-ab-ct-alphas118-nlo
240411-ab-ct-alphas118-nlo, version 1; 101 PDF members


In [4]:
# Get the values of alphas...
alphas_plus = API.theory_info_table(theory_db_id=theory_plus).loc["alphas"].iloc[0]
alphas_central = API.theory_info_table(theory_db_id=theory_mid).loc["alphas"].iloc[0]
alphas_min = API.theory_info_table(theory_db_id=theory_min).loc["alphas"].iloc[0]

# ... and make sure the alphas shift in both directions is symmetric
delta_alphas_plus = alphas_plus - alphas_central
delta_alphas_min = alphas_central - alphas_min
if abs(delta_alphas_min - delta_alphas_plus) > 1e-6:
    raise ValueError("alphas shifts in both directions is not symmetric")
else:
    alphas_step_size = delta_alphas_min

In [5]:
beta_tilde = np.sqrt(covmat_scaling_factor) * (alphas_step_size / np.sqrt(2)) * np.array([1, -1])
S_tilde = beta_tilde @ beta_tilde

delta_pluss = [(np.sqrt(covmat_scaling_factor) / np.sqrt(2)) * (
    prior_theorypreds_plus - prior_theorypreds_central) for prior_theorypreds_plus,prior_theorypreds_central in zip(prior_theorypreds_pluss,prior_theorypreds_centrals) ]

delta_minuss = [(np.sqrt(covmat_scaling_factor) / np.sqrt(2)) * (
    prior_theorypreds_minus - prior_theorypreds_central
) for prior_theorypreds_minus,prior_theorypreds_central in zip(prior_theorypreds_minuss,prior_theorypreds_centrals)]

betas = [[delta_plus, delta_minus] for delta_plus, delta_minus in zip(delta_pluss,delta_minuss)]
S_hats = [beta_tilde @ beta for beta in betas]

Ss = [np.outer(delta_plus, delta_plus) + np.outer(delta_minus, delta_minus) for delta_plus,delta_minus in zip(delta_pluss,delta_minuss)]
Ss = [pd.DataFrame(S, index=delta_minus.index, columns=delta_minus.index) for S,delta_minus in zip(Ss, delta_minuss)]

In [ ]:
theorypreds_fits = [API.group_result_table_no_table(**inps_central_fit).iloc[:, 2:] for inps_central_fit in inps_central_fits]

t0theoryid = API.t0theoryid(**theoryids_dicts[0])

# Experimental covariance matrix
Cs = [API.groups_covmat(
    use_t0=True,
    datacuts={"from_": "fit"},
    t0pdfset={"from_": "datacuts"},
    theoryid=t0theoryid.id,
    faketheoryid=t0theoryid.id,
    t0theoryid=t0theoryid.id,
    **common_dict
)for common_dict in common_dicts]

LHAPDF 6.4.0 loading all 496 PDFs in set 240715-ab-multict-fseed10290-tcm-nlo
240715-ab-multict-fseed10290-tcm-nlo, version 1; 496 PDF members
LHAPDF 6.4.0 loading all 496 PDFs in set 240715-ab-multict-fseed3345348918-tcm-nlo
240715-ab-multict-fseed3345348918-tcm-nlo, version 1; 496 PDF members


In [ ]:
# Different from the prediction of the mean PDF (i.e. replica0)
mean_predictions = [theorypreds_fit.mean(axis=1) for theorypreds_fit in theorypreds_fits ]

Xs = [np.zeros_like(C.values) for C in Cs]
for theorypreds_fit, X, mean_prediction in zip(theorypreds_fits, Xs, mean_predictions ):
    for i in range(theorypreds_fit.shape[1]):
        X += np.outer(
            (theorypreds_fit.iloc[:, i] - mean_prediction),
            (theorypreds_fit.iloc[:, i] - mean_prediction),
        )
    X *= 1 / theorypreds_fit.shape[1]

pseudodatas = [API.read_pdf_pseudodata(**common_dict) for common_dict in common_dicts]
dat_repss = [pd.concat(
    [i.pseudodata.reindex(prior_theorypreds_central.index) for i in pseudodata], axis=1
) for pseudodata,prior_theorypreds_central in zip(pseudodatas,prior_theorypreds_centrals)]

In [ ]:
invcovs = [np.linalg.inv(C + S) for C,S in zip(Cs, Ss)]
delta_T_tildes = [-S_hat @ invcov @ (mean_prediction - dat_reps.mean(axis=1)) for S_hat, invcov, mean_prediction,dat_reps  in zip(S_hats, invcovs, mean_predictions,dat_repss)]
P_tildes = [S_hat.T @ invcov @ X @ invcov @ S_hat + S_tilde - S_hat.T @ invcov @ S_hat for S_hat, invcov, X  in zip(S_hats, invcovs,Xs)]
preds = [alphas_central + delta_T_tilde for delta_T_tilde in delta_T_tildes]
uncs = [np.sqrt(P_tilde) for P_tilde in P_tildes]
tcm_alphas_results = []
for pred, unc in zip(preds, uncs):
    tcm_alphas_results.append([pred, unc])
    print(rf"Prediction for $\alpha_s$: {pred:.4f} ± {unc:.4f}")

In [ ]:
np.save("tcm_alphas_results", np.array(tcm_alphas_results))

In [ ]:
np.load("tcm_alphas_results.npy")